# OpenStreet Data Wrangling with Python and SQL
#### by Esther Xu

## Chosen Map Area
#### Arcadia, CA, United States
https://mapzen.com/data/metro-extracts/your-extracts/134f64cb12b1


I have chosen Arcadia for this project because this is a place where I live currently.I am more interested to explore the map of this area, and reveal some inconsistencies.

## Auditing and Cleaning the Data
After downloading and auditing the xml data of Arcadia area,and running the method ('audit_data(osm_file)') in auditingData.py. Then I noticed there are two problems in this representing data:
* Abbbreviated Street Names.                               
  Las Tunas Dr. => Las Tunas Drive                       
  850 S Baldwin Ave => 850 S Baldwin Avenue                         
  Duarte Rd => Duarte Road                                        

           
* Phone numbers had different formats.                      
  (626) 836-6888 => (626)836-6888                    
  +1(626) 305-9223 => (626)305-9223                    
  +1 626-301-7838 => (626)301-7838                         
  (625)574-YOGA => (625)574-9642                  

I used update_data_to_new_file() method in auditingData.py to creat a new file called update_arcadia.osm. In this file, I have made the following changes to the above problems:

First of all I changed all abbreviations of street type.

In addition, I changed phone representations to (626)999-9999.I standardized the phone number formatting by removing all spaces,hyphens,parenthesis,dot,and plus sign.Then I convert the letters to numbers and formatted phone with parenthesis and hyphens.

## Preparing for SQL Database
After auditing is complete the next step is to prepare the data to be inserted into a SQL database. To do so I parsed the elements in the OSM XML file, transformed them document format to tabular format, thus making it possible to write to .CSV files. These csv files can then easily be imported to a SQL database as tables.You can take a look at code in transferToSQL.py.

Eventually I built a SQL database and imported tables to this database from csv file. I used sqlite3 shell for this propose.The following these steps:          

** 1.create a new database by executing command:  **               

In [2]:
sqlite3 openstreet.db

**2.create tabales:**    

In [3]:
sqlite> .read create_db.sql                   
sqlite> .tables   

nodes       nodes_tags  ways        ways_nodes  ways_tags 

**3.import data:**  

In [5]:
sqlite> .import nodes.csv nodes
sqlite> .import ways.csv ways

sqlite> .import nodes_tags.csv nodes_tags
sqlite> delete from nodes_tags where id = 'id';

sqlite> .import ways_tags.csv ways_tags
sqlite> delete from ways_tags where id = 'id';

sqlite> .import ways_nodes.csv ways_nodes
sqlite> delete from ways_nodes where id = 'id';

## Querying SQL Database
In this section, some basic statitics about the data are presented.

### File Sizes
* arcadia.osm ................... 222.5 MB
* update_arcadia.osm ...... 225.1 MB
* nodes.csv ...................... 92.1 MB
* nodes_tags.csv .............. 0.25 KB
* ways.csv ........................ 6.3 MB
* ways_tags.csv ............... 20.2 MB
* ways_nodes.cv .............. 25.9 MB 
* openstreet.db ................ 126.1 MB 

### Number of Nodes

In [9]:
sqlite> select count(*) from nodes;

969735

### Number of Ways

In [11]:
sqlite> select count(*) from ways;    

90545

### Number of Unique Users

In [12]:
sqlite> select count(distinct(uid)) from (select uid from nodes union all select uid from ways);                 

247

### Top 10 Contributing Users

In [13]:
sqlite> select user,count(uid) as cnt from (select user,uid from nodes union all select user,uid from ways) group by user order by cnt desc limit 10;                  

nammala_labuildings,297999             
poornima_labuildings,185528                 
saikabhi_LA_imports,123392                
yurasi_import,114860                 
dannykath_labuildings,93623                 
upendra_labuilding,44168             
Jothirnadh_labuildings,41904              
karitotp_labuildings,37524                  
nikhil_imports,28863                             
schleuss_imports,26361 

### Number of users appearing only once (having 1 post)

In [14]:
sqlite> select count(*) from (select user, count(*) as cnt from (select user,uid from nodes union all select user,uid from ways) group by uid having cnt = 1);                                          

48

### The Distribution of Religion 

In [15]:
sqlite> SELECT a.value, COUNT(*) as cnt                  
   ...> FROM nodes_tags a,                    
   ...>      (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') b                 
   ...> WHERE a.id = b.id                              
   ...>   and a.key='religion'                   
   ...> GROUP BY a.value               
   ...> ORDER BY cnt DESC;      

christian,80

### Most Popular Cuisines

In [16]:
sqlite> SELECT a.value, COUNT(*) as cnt                 
   ...> FROM nodes_tags a,                         
   ...>      (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') b                  
   ...> WHERE a.id = b.id                        
   ...>   and a.key='cuisine'                  
   ...> GROUP BY a.value                    
   ...> ORDER BY cnt DESC;      

american,10                     
chinese,9                
italian,5              
mexican,5             
japanese,3            
pizza,3            
asian,2            
burger,2            
mediterranean,2              
sushi,2             
thai,2             
Cuban,1            
french,1            
international,1              
taiwanese,1    

## Conclusion
OpenStreetMap data is not perfect which can be modified by anyone.There are a lot human-made errors in the data.I just made my first step which was standarized street types,and phones numbers to make them more consistent and uniform. Then I transformed XML to CSV format and imported it into SQL database.Finally I answered some questions by SQL queries.

In this data analysis, all data have not been formatted and cleaned completely because it is so large. However, I parsed this data and corrected street types, post codes and phones formatting. 

To improve the quality of OpenStreeMap data is very important.For my perspective,there are two ways to do it.

* First of all,making some standards to retrict input by procedure so that improve the accuracy of the data.For example, entering phone number should be only this format:(xxx)xxx-xxxx. So that data can be easily used.

* Second, the human-made data is automatically compared with the data from professional organization through the procedure.                
              
Benefits                        
1.maximun benefit of overall map quality                      
2.Be used by more people                              

Anticipated Issues                 
1.difficult to obtain third-party data and verify the validiy of its data            
2.need a lot of money into the development of the program                  
